# Klasifikasi

In [1]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin, BaseEstimator
from collections import defaultdict
import numpy as np
import time

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\farha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv('Fix_Dataset/asuransi_clean.csv')

In [3]:
df.head()

,created_at,full_text,username,Produk,rate
0,2023-09-25,"['1', '5', 'juta', 'sist', 'kantor', 'sih', 'j...",Ditanyadia,AIA,2
1,2023-09-22,"['mohon', 'klarifikasi', 'dari', 'asuransi', '...",MuhTaufikRahmat,AIA,2
2,2023-09-21,"['indonesia', 'unit', 'link', '5', 'tips', 'se...",fiona99351131,AIA,2
3,2023-09-21,"['halo', 'bang', 'maaf', 'nimbrung', 'pesan', ...",fiayeee,AIA,2
4,2023-09-18,"['impactnya', 'rumah', 'sakit', 'rekan', 'reka...",filcadet,AIA,1


In [4]:
df['Produk'].value_counts().sort_values(ascending=True)

Produk
AIA           263
Prudential    302
BNI Life      540
Allianz       585
Manulife      715
Cigna         816
Name: count, dtype: int64

In [5]:
# encoding label kategori produk asuransi menggunakan dictionary

label = { 'AIA' : 0, 'Allianz' : 1, 'Prudential' : 2, 'BNI Life' : 3, 'Manulife' : 4, 'Cigna' : 5 }

df['label'] = df['Produk'].replace(label)

In [6]:
# melihat text yang memiliki konten null atau kosong
df['full_text'].isnull().sum()

0

In [7]:
# split data
X = df['full_text']
y = df['label']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# klassifikasi menggunakan model word2vec yang sudah di training sebelumnya menggunakan data corpus wikipedia
# load model word2vec
model_w2vec = Word2Vec.load('Word2vec/Model_wikipedia/idwiki_word2vec.model')

# Word2Vec

In [109]:
class MeanEmbeddingVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.vector_size = word2vec.vector_size

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec.wv[w] for w in word_tokenize(words) if w in self.word2vec.wv] or [np.zeros(self.vector_size)], axis=0)
            for words in X
        ])

## SVM

In [110]:
pipeline_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(model_w2vec)),
    ("classifier", LogisticRegression())
])

In [111]:
star_time = time.time()
pipeline_w2v.fit(X_train, y_train)
end_time = time.time()
print('Training time: ', end_time - star_time)

Training time:  1.4097445011138916


In [112]:
star_time = time.time()
y_pred_w2v = pipeline_w2v.predict(X_test)
end_time = time.time()
print('Prediction time: ', end_time - star_time)

Prediction time:  0.40686726570129395


In [113]:
# akurasi model
print("Akurasi Model Word2Vec: ", accuracy_score(y_test, y_pred_w2v))

Akurasi Model Word2Vec:  0.25116279069767444


## Save Model

In [114]:
import pickle

In [115]:
# save model
with open('Model/model_svm_w2v.pkl', 'wb') as model_file:
    pickle.dump(pipeline_w2v, model_file)

# Fast text dan TF-IDF

In [ ]:
from gensim.models import FastText
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
FastText_model = FastText.load('Word2vec/Model_wikipedia/idwiki_word2vec.model')

In [ ]:
# Penggabungan FastText dan TF-IDF
class FastTextEmbeddingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model):
        self.model = model
        self.dim = model.vector_size

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.model.wv[word] for word in word_tokenize(doc) if word in self.model.wv] or [np.zeros(self.dim)], axis=0)
            for doc in X
        ])

In [ ]:
pipeline_FT = Pipeline([
    ('features', FeatureUnion([
        ('FastText', FastTextEmbeddingTransformer(FastText_model)),
        ('tfidf', TfidfVectorizer()),
    ])),
    ('classifier', SVC(kernel='linear'))
])
    

In [ ]:
pipeline_FT.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('FastText',
                                                 FastTextEmbeddingTransformer(model=<gensim.models.word2vec.Word2Vec object at 0x000001ADB6C0C850>)),
                                                ('tfidf', TfidfVectorizer())])),
                ('classifier', SVC(kernel='linear'))])

In [ ]:
y_pred_FT = pipeline_FT.predict(X_test)

In [ ]:
print('Accuracy: ', accuracy_score(y_test, y_pred_FT))

Accuracy:  0.9085271317829458
